In [24]:
import pyvista as pv
import meshio
import numpy as np
import psycopg2
import hdf5storage
import h5py
import pickle
import logging
from sys import getsizeof
from dotenv import dotenv_values

pv.global_theme.transparent_background = True


config = dotenv_values("../.env")
# Filepaths for script
initMeshPath = config["currentDirectory"] +"meshes/fullAssemblyContiguous/fullFishContiguous.msh"
tempMeshPath = config["currentDirectory"] + "data/visualizations/tempMeshes/"
outputPath = config["currentDirectory"] + "data/visualizations/controlExperiments/"


In [25]:
# Helper functions for working with database
def get_db_connection():
    conn = psycopg2.connect(
        dbname='simDB',
        user='user',
        password='password',
        host='localhost',
        port='5432'
    )
    return conn

def fetch_trial_data_singleTimestep(conn, trial_id, n=1, start_timestep=0, num_timesteps=500):
    try:
        cur = conn.cursor()
        
        # Fetch data for the given trial_id starting from start_timestep
        query = '''
        SELECT timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat
        FROM simulation_data
        WHERE trial_id = %s AND timestep >= %s AND timestep < %s
        ORDER BY timestep ASC;
        '''
        cur.execute(query, (trial_id, start_timestep, start_timestep + 1))
        rows = cur.fetchall()
        
        # Deserialize data
        data = []
        for row in rows:
            timestep, simulation_time, input_data_bin, output_data_bin, state_data_bin, y_ref_bin, x_hat_bin = row
            input_data = np.array(pickle.loads(input_data_bin))
            output_data = np.array(pickle.loads(output_data_bin))
            state_data = np.array(pickle.loads(state_data_bin))
            y_ref = np.array(pickle.loads(y_ref_bin))
            x_hat = np.array(pickle.loads(x_hat_bin))
            data.append((timestep, simulation_time, input_data, output_data,state_data, y_ref, x_hat))
        
        cur.close()
        return data
    except Exception as e:
        print(f"Error fetching data for trial_id {trial_id}: {e}")
        return None
    


In [26]:
# Trial Parameters
trial_ids = [150, 151]
timesteps = 1500


In [27]:
### Get offsets for reference trajectory
# Load data from database
conn = get_db_connection()
data = fetch_trial_data_singleTimestep(conn, trial_ids[0], 1, start_timestep=0, num_timesteps=1)
conn.close()
timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
# Get the reference trajectory
refOffsets = np.copy(output_data)


In [28]:
###### Generate mp4 of fish swimming in isometric view along with reference trajectory for specified trials
for trial_id in trial_ids:
    # create plotter object
    plotter = pv.Plotter()
    # Open a mp4
    plotter.open_movie(outputPath + f"Trial{trial_id}_isoView.mp4", framerate = 100, quality=7)
    # Iterate through timesteps and add frames
    for i in range(timesteps):
        print(f"Trial {trial_id}, Timestep {i}")
        ### Make surface from desired reference trajectory
        # Load data from database
        conn = get_db_connection()
        data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
        conn.close()
        timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        # Check shape of reference trajectory
        uncenteredRefTraj=y_ref + refOffsets
        # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
        x_refs = uncenteredRefTraj[::2]
        z_refs = uncenteredRefTraj[1::2]
        referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
        # print(np.shape(referenceCurve))
        path = referenceCurve
        offsetPath = np.copy(path)
        offsetPath[:, 1] = offsetPath[:, 1] - 200
        #### Draw Surface for reference trajectory
        # Number of samples (in depth dir) and number of points along centerline 
        nsamples = 5
        ntraces = 20  

        # Define the Z spacing of your 2D section
        z_spacing = 50
        # Create structured points draping down from the path
        points = np.repeat(path, nsamples, axis=0)
        # repeat the Z locations across
        tp = np.arange(0, z_spacing * nsamples, z_spacing)
        # Offset the Z locations 
        tp = path[:, 1][:, None] - tp
        points[:, 1] = tp.ravel()
        #Make a StructuredGrid from the structured points
        grid = pv.StructuredGrid()
        grid.points = points
        grid.dimensions = nsamples, ntraces, 1

        # Add the data array - note the ordering
        grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")

        # load mesh from .vtu file in tempMeshPath
        mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{i}.vtu")
        # add fish mesh to plotter
        plotter.add_mesh(mesh, show_scalar_bar=False, name = 'mesh') # add the fish body
        plotter.add_mesh(grid, clim=[-1, 1],show_scalar_bar=False, name = 'reference', opacity=0.5)
        # add reference trajectory to plotter with thickness of 5 and large markers
        plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name = 'refPath', opacity=1, render_points_as_spheres=True, point_size=10)
        # Set the camera position
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 40#90 #
        plotter.camera.elevation = 50#0 #50
        plotter.enable_parallel_projection()

        # write frame to gif
        plotter.write_frame()
    # Close the gif
    plotter.close()

    



Trial 150, Timestep 0
Trial 150, Timestep 1
Trial 150, Timestep 2
Trial 150, Timestep 3
Trial 150, Timestep 4
Trial 150, Timestep 5
Trial 150, Timestep 6
Trial 150, Timestep 7
Trial 150, Timestep 8
Trial 150, Timestep 9
Trial 150, Timestep 10
Trial 150, Timestep 11
Trial 150, Timestep 12
Trial 150, Timestep 13
Trial 150, Timestep 14
Trial 150, Timestep 15
Trial 150, Timestep 16
Trial 150, Timestep 17
Trial 150, Timestep 18
Trial 150, Timestep 19
Trial 150, Timestep 20
Trial 150, Timestep 21
Trial 150, Timestep 22
Trial 150, Timestep 23
Trial 150, Timestep 24
Trial 150, Timestep 25
Trial 150, Timestep 26
Trial 150, Timestep 27
Trial 150, Timestep 28
Trial 150, Timestep 29
Trial 150, Timestep 30
Trial 150, Timestep 31
Trial 150, Timestep 32
Trial 150, Timestep 33
Trial 150, Timestep 34
Trial 150, Timestep 35
Trial 150, Timestep 36
Trial 150, Timestep 37
Trial 150, Timestep 38
Trial 150, Timestep 39
Trial 150, Timestep 40
Trial 150, Timestep 41
Trial 150, Timestep 42
Trial 150, Timestep 4

In [29]:
###### Generate mp4 of fish swimming in Top down view along with reference trajectory for specified trials
for trial_id in trial_ids:
    # create plotter object
    plotter = pv.Plotter()
    # Open a mp4
    plotter.open_movie(outputPath + f"Trial{trial_id}_topView.mp4", framerate = 100, quality=7)
    # Iterate through timesteps and add frames
    for i in range(timesteps):
        print(f"Trial {trial_id}, Timestep {i}")
        ### Make surface from desired reference trajectory
        # Load data from database
        conn = get_db_connection()
        data = fetch_trial_data_singleTimestep(conn, trial_id, 1, start_timestep=i, num_timesteps=1)
        conn.close()
        timestep, simulation_time, input_data, output_data, state_data, y_ref, x_hat = data[0]
        # Check shape of reference trajectory
        uncenteredRefTraj=y_ref + refOffsets
        # Turn reference trajectory from 40x1 to 20x2 array by taking every other point 
        x_refs = uncenteredRefTraj[::2]
        z_refs = uncenteredRefTraj[1::2]
        referenceCurve = np.column_stack((x_refs, np.zeros(np.size(x_refs)), z_refs))
        # print(np.shape(referenceCurve))
        path = referenceCurve
        offsetPath = np.copy(path)
        offsetPath[:, 1] = offsetPath[:, 1] - 200
        #### Draw Surface for reference trajectory
        # Number of samples (in depth dir) and number of points along centerline 
        nsamples = 5
        ntraces = 20  

        # Define the Z spacing of your 2D section
        z_spacing = 50
        # Create structured points draping down from the path
        points = np.repeat(path, nsamples, axis=0)
        # repeat the Z locations across
        tp = np.arange(0, z_spacing * nsamples, z_spacing)
        # Offset the Z locations 
        tp = path[:, 1][:, None] - tp
        points[:, 1] = tp.ravel()
        #Make a StructuredGrid from the structured points
        grid = pv.StructuredGrid()
        grid.points = points
        grid.dimensions = nsamples, ntraces, 1

        # Add the data array - note the ordering
        grid["values"] = np.ones((nsamples, ntraces), dtype=np.float64).ravel(order="F")

        # load mesh from .vtu file in tempMeshPath
        mesh = pv.read(tempMeshPath + f"Trial{trial_id}_step{i}.vtu")
        # add fish mesh to plotter
        plotter.add_mesh(mesh, show_scalar_bar=False, name = 'mesh') # add the fish body
        plotter.add_mesh(grid, clim=[-1, 1],show_scalar_bar=False, name = 'reference', opacity=0.5)
        # add reference trajectory to plotter with thickness of 5 and large markers
        plotter.add_mesh(pv.PolyData(offsetPath), color='orange', line_width=5, name = 'refPath', opacity=1, render_points_as_spheres=True, point_size=10)
        # Set the camera position
        plotter.camera_position = 'yz'
        plotter.camera.roll = 180
        plotter.camera.azimuth = 90 #
        plotter.camera.elevation = 90 #50
        plotter.enable_parallel_projection()

        # write frame to gif
        plotter.write_frame()
    # Close the gif
    plotter.close()

    



Trial 150, Timestep 0
Trial 150, Timestep 1
Trial 150, Timestep 2
Trial 150, Timestep 3
Trial 150, Timestep 4
Trial 150, Timestep 5
Trial 150, Timestep 6
Trial 150, Timestep 7
Trial 150, Timestep 8
Trial 150, Timestep 9
Trial 150, Timestep 10
Trial 150, Timestep 11
Trial 150, Timestep 12
Trial 150, Timestep 13
Trial 150, Timestep 14
Trial 150, Timestep 15
Trial 150, Timestep 16
Trial 150, Timestep 17
Trial 150, Timestep 18
Trial 150, Timestep 19
Trial 150, Timestep 20
Trial 150, Timestep 21
Trial 150, Timestep 22
Trial 150, Timestep 23
Trial 150, Timestep 24
Trial 150, Timestep 25
Trial 150, Timestep 26
Trial 150, Timestep 27
Trial 150, Timestep 28
Trial 150, Timestep 29
Trial 150, Timestep 30
Trial 150, Timestep 31
Trial 150, Timestep 32
Trial 150, Timestep 33
Trial 150, Timestep 34
Trial 150, Timestep 35
Trial 150, Timestep 36
Trial 150, Timestep 37
Trial 150, Timestep 38
Trial 150, Timestep 39
Trial 150, Timestep 40
Trial 150, Timestep 41
Trial 150, Timestep 42
Trial 150, Timestep 4